In [9]:
import cv2
from os import listdir
from os.path import isfile, join
import glob
import numpy as np
import time

In [10]:
def GMM(pixel, mus, sigs, wts):
    F = 0
    assigned = False
    dist = np.zeros(4)
    for i in range(0,4):
        dist[i] = pixel - mus[i]
        if(wts[i]!=0 and ( pixel < (mus[i] + 2.5*sigs[i]) and pixel > (mus[i] - 2.5*sigs[i]))):
            #updateparams
            wts = [(1-0.1)*(wts[j])+0.1 if(j==i) else (1-0.1)*(wts[j]) for j in range(0,4)] #update weights
            rh=(np.exp(-np.power(pixel - mus[i], 2) / (2 * np.power(sigs[i], 2))))*(0.399/sigs[i])*0.1 #rho calculation
            mus[i] = (1-rh)*(mus[i])+(rh*pixel) # update means
            sigs[i]=(1-rh)*(sigs[i])+(rh*np.power(pixel-mus[i],2)) #update variance   
            assigned = True
#             print("mus:",mus[i])
#             print("updated") 
            break
        elif(wts[i]==0):
            # form new gaussian
            for j in range(0,4):
                if(j==i):
                    wts[j]=0.1
                else:
                    wts[j]=(0.9)*wts[j]
                                
            #wts[i+1] = 0.1
            mus[i] = pixel
            sigs[i]= 19
            assigned = True
            break
    
    if (assigned == False):
        k = np.argmin(dist)
        wts = [(1-0.1)*(wts[j])+0.1 if(j==k) else (1-0.1)*(wts[j]) for j in range(0,4)] #update weights
        rh=(np.exp(-np.power(pixel - mus[k], 2) / (2 * np.power(sigs[k], 2))))*(0.399/sigs[k])*0.1 #rho calculation
        mus[k] = (1-rh)*(mus[k])+(rh*pixel) # update means
        sigs[k]=(1-rh)*(sigs[k])+(rh*np.power(pixel-mus[k],2))
    
    indices = np.argsort(-(wts/sigs),axis=0)                    
    tot = 0
    j=0
    while(tot<=0.06):
        tot = tot + wts[indices[j]]
        j=j+1
    if i in indices[j:]:
        # Assign curret pixel to foreground class
        F = 255
    
    return mus,sigs,wts, F

In [11]:
def init_params(frames, means, var, wts):
    means[0] = frames[:,0]
    var[0]= np.array([19 for i in range(var[0].shape[0])])
    wts[0] = np.array([1 for i in range(var[0].shape[0])])

In [12]:
# Function to extract frames
def FrameCapture(path):
    vidObj = cv2.VideoCapture(path)
    count = 0
    success = 1
    frames = np.zeros((240,352,999))
    while success:
        # vidObj object calls read
        # function extract frames
        success, image = vidObj.read()
        if(success):
            # Saves the frames with frame-count
            img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            frames[:,:,count] = img_gray
            cv2.imwrite("/home/ajey/Desktop/ELL784/Assignment1/Original_frames/frame_"+str(count)+".jpg",img_gray)
            count += 1
    return frames

In [ ]:
if __name__ == '__main__':
    start = time.time()
    numFrames = 130
    frames = np.zeros((240,352,numFrames))
    cwd = "/home/ajey/Desktop/ELL784/Assignment1/umcp.mpg"
    frames = FrameCapture(cwd)
    out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*'XVID'), 30, (352,240), 0)
    back = cv2.VideoWriter("back.avi", cv2.VideoWriter_fourcc(*'XVID'), 30, (352,240), 0)
    output = np.zeros((frames.shape[0]*frames.shape[1], frames.shape[2]))
    frames = frames.reshape((frames.shape[0]*frames.shape[1], frames.shape[2]))
    background = frames
    # frames.shape #The reshaped array is of size (84480, 999). Here, each row corresponds to the entire "Pixel process" of one Pixel
    means = np.zeros((4,84480))
    var = np.zeros((4,84480))
    wts = np.zeros((4,84480))
    init_params(frames, means, var, wts)
    F = 0 # Flag for classification (0 if background, 1 if foreground)
    for i in range(0,84480): #For every pixel
        for j in range(0,numFrames): #For every frame
            means[:,i],var[:,i],wts[:,i], F = GMM(frames[i,j], means[:,i], var[:,i], wts[:,i])
            output[i,j] = F
            if F!=0:
                background[i,j] = np.mean(frames[i,:])
                background[i:i+5,j:j+5] = np.mean(frames[i,:])
                background[i-5:i,j-5:j] = np.mean(frames[i,:])
        print(i)

In [16]:
    for i in range(numFrames):
        outframe = output[:,i].reshape((240,352))
        outframe = cv2.resize(outframe, (352,240))
        backframe = background[:,i].reshape((240,352))
        backframe = cv2.resize(backframe, (352,240))
        out.write(np.uint8(outframe))
        back.write(np.uint8(backframe))
        cv2.imwrite("/home/ajey/Desktop/ELL784/Assignment1/background_frames/output_"+str(i)+".jpg",outframe)
        cv2.imwrite("/home/ajey/Desktop/ELL784/Assignment1/back_minus/"+str(i)+".jpg",backframe)
    out.release()
    back.release()
    end = time.time()
    print((end-start)/60)

14.768151140213012
